#### Problem Statement:

### **Predicting High-Profit Sales Transactions Using Machine Learning**

---

#### Objective 
***
The goal of this project is to develop a machine learning model that can predict whether a sales transaction will yield a high profit (defined as a profit greater than $10,000). This predictive model will help the business prioritize and strategize its sales efforts to maximize profitability

#### Data Description
***
The dataset contains 500,000 sales records with the following attributes:

- Region: The geographical region where the sale was made.
- Country: The country where the sale was made.
- Item Type: The type of item sold (e.g., Fruits, Clothes, Meat, etc.).
- Sales Channel: The sales channel used (Online or Offline).
- Order Priority: The priority of the order (High, Medium, Low, Critical).
- Order Date: The date when the order was placed.
- Order ID: A unique identifier for the order.
- Ship Date: The date when the order was shipped.
- Units Sold: The number of units sold.
- Unit Price: The price per unit.
- Unit Cost: The cost per unit.
- Total Revenue: The total revenue from the sale.
- Total Cost: The total cost of the sale.
- Total Profit: The total profit from the sale